---
# Medtronic(Soft Robotics)
# MCEN 4045, Fall 2021
# Tuesday November 16, 2021

The following command downloads the NDI Combined API C Interface Library from PyPi with out the need to complie the github repo found here: https://github.com/PlusToolkit/ndicapi

In [17]:
!pip install ndicapi

---
## Library import
We are referencing the code we found in the repo
secret sauce: https://github.com/PlusToolkit/ndicapi/blob/master/Applications/ndiBasicExample.py

In [18]:
import ndicapy # import is strange

In [19]:
from ndicapy import (
    ndiDeviceName, ndiProbe, NDI_OKAY,
    ndiOpen, ndiClose, ndiCommand, ndiGetError,
    ndiErrorString, NDI_115200,
    NDI_8N1, NDI_NOHANDSHAKE,
)

In [24]:
from IPython.display import clear_output

--- 
## Port Setup

In [20]:
# Number ports to check
MAX_SERIAL_PORTS = 20

name = ''
for port_no in range(MAX_SERIAL_PORTS):
    name = ndiDeviceName(port_no)
    
    if not name:
        continue
        
    result = ndiProbe(name) #Returns 257 if port not found, otherwise returns 1
    
    if result == NDI_OKAY: #NDI_OKAY == 0
        print(port_no)
        break
        
if result != NDI_OKAY:
    raise IOError(
        'Could not find any NDI device in '
        '{} serial port candidates checked. '
        'Please check the following:\n'
        '\t1) Is an NDI device connected to your computer?\n'
        '\t2) Is the NDI device switched on?\n'
        '\t3) Do you have sufficient privilege to connect to '
        'the device? (e.g. on Linux are you part of the "dialout" '
        'group?)'.format(MAX_SERIAL_PORTS)
    )

2


--- 
## Device Setup

In [21]:
device = ndiOpen(name) #seems like this command starts communication for API 
if not device:
    raise IOError(
        'Could not connect to NDI device found on '
        '{}'.format(name)
    )
print(device)

<polaris object 000001CA5C6C4BF0, COM3>


--- 
## Command Send Check

In [22]:
#Ensures the system configuuration was determine successfully
reply = ndiCommand(device, 'INIT:') 
error = ndiGetError(device)
if reply.startswith('ERROR') or error != NDI_OKAY:
    raise IOError(
        'Error when sending command: '
        '{}'.format(ndiErrorString(error))
    )
print(reply)

OKAY


In [23]:
# Sets serial communication settings
reply = ndiCommand(
    device,
    'COMM:{:d}{:03d}{:d}'.format(NDI_115200, NDI_8N1, NDI_NOHANDSHAKE)
)
print(reply)

OKAY


--- 
## Insert Commands HERE

Sensor part #:610059 T6F0-L00054

Puck Part# : 610066 T6D0-S00914
$\therefore$

In [25]:
num_setup_complete = 0
while(num_setup_complete < 2):
    To_Free = ndiCommand(device, 'PHSR:01')

    if(To_Free == '00'):
        To_Initialized = ndiCommand(device, 'PHSR:02')

        if(To_Initialized == '00'):
            To_enable = ndiCommand(device, 'PHSR:03')
            print(To_enable[2:4])
            if(To_enable == '00'):
                break
            else:
                ndiCommand(device, 'PENA:{}{}'.format(To_enable[2:4], 'D'))
                print("Success:")
                print(ndiCommand(device, 'PHINF:{}{}'.format(To_enable[2:4], '0004')))
                num_setup_complete += 1
        else:
            #There needs to be ports to be initialized
            ndiCommand(device, 'PINIT:{}'.format(To_Initialized[2:4]))
    else:
        #ports need to be freed
        print("free ports please")

0A
Success:
610059   T6F0-L00054
0B
Success:
610066   T6D0-S00914


In [26]:
reply4 = ndiCommand(device, 'TSTART:')
print(reply4)

OKAY


In [28]:
while(True):
    reply5 = ndiCommand(device, 'TX:')
    clear_output(wait=True)
    print(reply5)
print(len(reply5))

020A+00083+02303+09730-00115+013641-004923-030312+013550002003F000178C6
0B+00190-05956+08006+00618-003424-017684-006547+026020002003D000178C6
0000


KeyboardInterrupt: 

In [ ]:
reply = ndiCommand(device, 'TSTOP:') 
print(reply)

--- 
## Terminate Communication

In [ ]:
ndiClose(device)

--- 
## API commands
Secret sauce #2: https://github.com/lara-unb/APIs-NDI-Digital/blob/master/Aurora_API_Guide.pdf

In [ ]:
pip install IPython